
# SBS 3D Video Generation Pipeline

This notebook outlines the process of converting a monocular video into a side-by-side (SBS) 3D video.



## Setup and Preparation

Import necessary libraries and define the input video path.

Personally, I opt for the file structure `datasets/d{index}/[set of input/output folder for frames]`

In [ ]:
import timeit
import os
import subprocess

dataset = 1
# Define the path to the input video
dataset_directory = f'datasets/d{dataset}/'
input_video_path = dataset_directory + 'test.mp4'
final_video_filename = dataset_directory + 'test_SBS.mp4'

## Import and Setup Depth Anything Project

Run depth anything model for video input, specify if wishing to include depth model locally

In [ ]:
local_path = False
local_path_directory = "your_local_directory"
current_working_directory = os.getcwd()

In [ ]:
if local_path:
    local_path_directory = r"depth_models"
    # Clone the repository
    os.system("git clone https://github.com/LiheYoung/Depth-Anything")
    # Change directory to the cloned repository
    os.chdir("Depth-Anything")
    # Create a Conda environment named 'depth-anything' with Python 3.11
    os.system("conda create -n depth-anything python=3.11")
    # Activate the Conda environment
    os.system("conda activate depth-anything")
    # Install PyTorch and other dependencies
    os.system("conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia")
    # Install additional Python packages from requirements.txt
    os.system("pip install -r requirements.txt")
    # Print a message to indicate successful setup
    print("Project setup complete.")

In [ ]:
# if not local path, specify remote path for depth-anything
remote_path = r"..\Depth-Anything"
if not local_path: local_path_directory = remote_path
print(f"Path set {local_path_directory}")

In [ ]:
def checkPath(path):
    if os.path.exists(path): print(f"The file '{path}' exists.")
    else: print(f"The file '{path}' does not exist.")

In [ ]:
rgbd_frames = dataset_directory + 'rgbd_in/'
os.makedirs(rgbd_frames, exist_ok=True)

print(f"dataset directory {dataset_directory}\n video dire {input_video_path}\n outdir {dataset_directory}")
print(f"local path {local_path_directory}")

# Check if the file exists
# file_path = os.path.join(local_path_directory, "run_depth_only.py")
file_path = os.path.join(os.getcwd(), "run_depth_only.py")

# Get the current working directory
current_working_directory = os.getcwd()

# Define the full paths to input video and dataset directory
input_video_path_full = os.path.join(current_working_directory, input_video_path)
dataset_directory_full = os.path.join(current_working_directory, dataset_directory)
checkPath(input_video_path_full)
checkPath(dataset_directory_full)


os.chdir(local_path_directory)
!python {local_path_directory}/run_depth_only.py --encoder vitl --video-path {input_video_path_full} --outdir {dataset_directory_full}


## Extract Frames from Video

Use ffmpeg to extract frames from the input color video and depth video.


In [ ]:
# switch directory back
output_depth_video = dataset_directory + "test_video_depth.mp4"
# extract audio from video
output_audio = "output_audio.aac"
!ffmpeg -i {input_video_path} -vn -acodec copy {output_audio}
os.chdir(current_working_directory)
print(f"Changed directory to {current_working_directory}")

In [ ]:
# Create directory if non-existent
output_frames_path = dataset_directory + 'rgbd_in/frame%d.jpg'
output_dir = os.path.dirname(output_frames_path)
os.makedirs(output_dir, exist_ok=True)
print(f"output_frames_path: {output_frames_path}, output_dir: {output_dir}")
print(f"video {input_video_path}")
# execute ffmpeg command for color
!ffmpeg -i {input_video_path} -q:v 2 {output_frames_path} 
output_frames_path = dataset_directory + 'rgbd_in/frame%d.png' # for depth
!ffmpeg -i {output_depth_video} -q:v 2 {output_frames_path}



## Image Preprocessing

Rename and pair color and depth images as needed. Run the script or run the function inside notebook.


In [ ]:
rgbd_frames = dataset_directory + 'rgbd_in/'
# Example for renaming images (adjust according to your script)
!python sbs_rename_directory.py {rgbd_frames}

In [ ]:
import os
import re

def get_frame_number(filename):
    match = re.search(r"frame(\d+)_", filename)
    if match:
        return int(match.group(1))
    else:
        raise ValueError(f"Invalid filename format: {filename}")
        
def rename_files(source_dir):
    os.makedirs(source_dir, exist_ok=True)

    # Process color images
    color_files = sorted([f for f in os.listdir(source_dir) if f.startswith("frame") and f.endswith(".jpg")], 
                         key=lambda x: int(x.split("frame")[1].split(".")[0]))
    counter = 1
    for filename in color_files:
        new_name = f"color{counter}.jpg"
        os.rename(os.path.join(source_dir, filename), os.path.join(source_dir, new_name))
        counter += 1
    print(f"Renamed {counter} color files in {source_dir}.")

    # Process depth images
    depth_files = sorted([f for f in os.listdir(source_dir) if f.startswith("frame") and f.endswith(".png")], 
                         key=lambda x: int(x.split("frame")[1].split(".")[0]))
    counter = 1
    for filename in depth_files:
        new_name = f"depth{counter}.png"
        os.rename(os.path.join(source_dir, filename), os.path.join(source_dir, new_name))
        counter += 1
    print(f"Renamed {counter} depth files in {source_dir}.")
    
source_dir = dataset_directory + "rgbd_in/"
rename_files(source_dir)


## Generate Stereo Views

Run the script to generate left and right eye views or run the function inside notebook.


In [ ]:
stereo_input_dir =  dataset_directory + "rgbd_in/"
stereo_output_dir = dataset_directory + "stereo_out_frames/"
os.makedirs(stereo_input_dir, exist_ok=True)
os.makedirs(stereo_output_dir, exist_ok=True)

!python sbs_generate_stereoviews.py {stereo_input_dir} {stereo_output_dir}

In [ ]:
import os
import cv2
import numpy as np
import os
import concurrent.futures

def shift_pixels(image, depth_map, direction, scale_factor, shift_threshold, frame_name, output_dir):
    height, width = image.shape[:2]
    disparity_map = (depth_map.astype(np.float32) * scale_factor).astype(np.int32)

    # Generate mesh for coordinates
    x_coords, y_coords = np.meshgrid(np.arange(width), np.arange(height))

    # Calculate new x coordinates
    new_x_coords = x_coords + (disparity_map * direction)

    # Initialize shifted image with zeros (black)
    shifted_image = np.zeros_like(image)

    # Calculate valid mask where new x coordinates are within image bounds
    valid_mask = (new_x_coords >= 0) & (new_x_coords < width)

    # Apply valid shifts to shifted_image
    valid_y, valid_x = np.where(valid_mask)
    new_valid_x_coords = new_x_coords[valid_y, valid_x]
    shifted_image[valid_y, new_valid_x_coords] = image[valid_y, valid_x]

    # Find significant shifts where disparity is greater than or equal to the shift threshold
    significant_mask = (np.abs(disparity_map) >= shift_threshold) & valid_mask
    significant_y, significant_x = np.where(significant_mask)

    # Correct new_x based on direction for significant pixels
    significant_new_x = significant_x + (disparity_map[significant_y, significant_x] * direction)

    # Keep only the significant shifts that are within bounds
    in_bounds_mask = (significant_new_x >= 0) & (significant_new_x < width)
    significant_new_x = significant_new_x[in_bounds_mask]
    significant_y = significant_y[in_bounds_mask]

    # Convert 2D coordinates to linear indices using new_x for the x-coordinate
    significant_shift_indexes = np.ravel_multi_index((significant_y, significant_new_x), dims=(height, width))
    # np.save(os.path.join(output_dir, f"{frame_name}_{direction}_significant_shifts.npy"), significant_shift_indexes)
    
    return shifted_image, significant_shift_indexes


# The rest of the process_image_pair function and the execution code remain unchanged.
def process_image_pair(args):
    color_image_path, depth_image_path, output_dir, frame_number, scale_factor, shift_threshold = args
    print(f"Processing frame {frame_number}")
    print(f"Processing frame {frame_number}")

    # Load the color image and depth map
    color_image = cv2.imread(color_image_path)
    depth_map = cv2.imread(depth_image_path, cv2.IMREAD_UNCHANGED)

    # Check if the images are loaded correctly
    if color_image is None:
        raise ValueError(f"Color image for frame {frame_number} could not be loaded from {color_image_path}")
    if depth_map is None:
        raise ValueError(f"Depth image for frame {frame_number} could not be loaded from {depth_image_path}")

    # Convert depth_map to grayscale if necessary
    if depth_map.ndim == 3:
        depth_map = cv2.cvtColor(depth_map, cv2.COLOR_BGR2GRAY)

    frame_name = os.path.splitext(os.path.basename(color_image_path))[0]
    left_eye_image, left_mask = shift_pixels(color_image, depth_map, 1, scale_factor, shift_threshold, frame_name, output_dir)
    right_eye_image, right_mask = shift_pixels(color_image, depth_map, -1, scale_factor, shift_threshold, frame_name, output_dir)

    # Check if the shifted images are valid
    if left_eye_image is None:
        raise ValueError(f"Left eye image processing failed for frame {frame_number}")
    if right_eye_image is None:
        raise ValueError(f"Right eye image processing failed for frame {frame_number}")

    # Ensure output directories exist
    left_eye_dir = os.path.join(output_dir, 'leftEye')
    right_eye_dir = os.path.join(output_dir, 'rightEye')
    os.makedirs(left_eye_dir, exist_ok=True)
    os.makedirs(right_eye_dir, exist_ok=True)

    # Save the processed images
    left_eye_output_path = os.path.join(left_eye_dir, f'leftEye{frame_number}.jpg')
    right_eye_output_path = os.path.join(right_eye_dir, f'rightEye{frame_number}.jpg')
    left_eye_mask_path = os.path.join(left_eye_dir, f'leftEyeMask{frame_number}.npy')
    right_eye_mask_path = os.path.join(right_eye_dir, f'rightEyeMask{frame_number}.npy')
    if not cv2.imwrite(left_eye_output_path, left_eye_image):
        raise ValueError(f"Failed to save left eye image for frame {frame_number} at {left_eye_output_path}")
    if not cv2.imwrite(right_eye_output_path, right_eye_image):
        raise ValueError(f"Failed to save right eye image for frame {frame_number} at {right_eye_output_path}")
    # if not np.save(left_eye_mask_path, left_mask):
    #     raise ValueError(f"Failed to save left eye image for frame {frame_number} at {left_eye_output_path}")
    # if not np.save(right_eye_mask_path, right_mask):
    #     raise ValueError(f"Failed to save right eye image for frame {frame_number} at {right_eye_output_path}")
    np.save(left_eye_mask_path, left_mask)
    np.save(right_eye_mask_path, right_mask)
    print(f"Saved processed images for frame {frame_number}")

def parallel_process_images(input_directory, output_directory, scale_factor, shift_threshold):
    os.makedirs(output_directory, exist_ok=True)

    color_images = sorted([f for f in os.listdir(input_directory) if f.startswith('color')])
    depth_images = sorted([f for f in os.listdir(input_directory) if f.startswith('depth')])
    frame_numbers = range(1, len(color_images))  # Assuming frame numbering starts at 1

    # Prepare arguments for each call to process_image_pair
    tasks = [(os.path.join(input_directory, color_images[i-1]),
              os.path.join(input_directory, depth_images[i-1]),
              output_directory, i, scale_factor, shift_threshold) for i in frame_numbers]

    # Using ThreadPoolExecutor to process images in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count() - 1) as executor:
        futures = [executor.submit(process_image_pair, task) for task in tasks]
        
        # Optionally, handle exceptions and results
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # Or print result if your function returns anything
            except Exception as e:
                print(f"An error occurred: {e}")
        # list(tqdm(concurrent.futures.as_completed(futures), total=len(futures)))
    print("Image processing completed.")


# Setup directories
input_directory = f"{dataset_directory}/rgbd_in"
output_directory = f"{dataset_directory}/stereo"
os.makedirs(output_directory, exist_ok=True)
scale_factor = 0.1
shift_threshold = 1
# Call the parallel processing function
parallel_process_images(input_directory, output_directory, scale_factor, shift_threshold)

print("Image processing completed.")


In [ ]:
%time

import cv2
import numpy as np
import os

def process_images(input_dir, output_dir, scale_factor):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    color_images = sorted([f for f in os.listdir(input_dir) if f.startswith('color')])
    depth_images = sorted([f for f in os.listdir(input_dir) if f.startswith('depth')])

    for color_image_path, depth_image_path in zip(color_images, depth_images):
        color_image = cv2.imread(os.path.join(input_dir, color_image_path))
        depth_map = cv2.imread(os.path.join(input_dir, depth_image_path), cv2.IMREAD_GRAYSCALE)

        if color_image is None:
            print(f"Error: Color image not found at {os.path.join(input_dir, color_image_path)}")
            continue

        if depth_map is None:
            print(f"Error: Depth map not found at {os.path.join(input_dir, depth_image_path)}")
            continue

        # Function to shift pixels based on depth map
        def shift_pixels(image, depth_map, direction):
            shifted_image = np.zeros_like(image)
            for y in range(image.shape[0]):
                for x in range(image.shape[1]):
                    disparity = calculate_disparity(depth_map[y, x])
                    new_x = x + disparity * direction
                    if 0 <= new_x < image.shape[1]:
                        shifted_image[y, new_x] = image[y, x]
            return shifted_image

        # Calculate disparity (example function, adjust as needed)
        def calculate_disparity(depth_value):
            # Simple linear mapping, adjust the scale factor as needed
            return int(depth_value * scale_factor)

        # Create left and right eye images
        left_eye_image = shift_pixels(color_image, depth_map, 1)
        right_eye_image = shift_pixels(color_image, depth_map, -1)

        frame_number = color_image_path.split('color')[1].split('.')[0]
        
        if not os.path.exists(os.path.join(output_dir,'leftEye')):
            os.makedirs(os.path.join(output_dir,'leftEye'))
        if not os.path.exists(os.path.join(output_dir,'rightEye')):
            os.makedirs(os.path.join(output_dir,'rightEye'))
        
        left_eye_output_path = os.path.join(output_dir, f'leftEye/leftEye{frame_number}.jpg')
        right_eye_output_path = os.path.join(output_dir, f'rightEye/rightEye{frame_number}.jpg')

        # Save the left and right eye images
        cv2.imwrite(left_eye_output_path, left_eye_image)
        cv2.imwrite(right_eye_output_path, right_eye_image)

        print(f"Processed frame {frame_number}.")

# Example usage
stereo_input_dir =  dataset_directory + "rgbd_in/"
stereo_output_dir = dataset_directory + "stereo_out_frames/"
os.makedirs(stereo_input_dir, exist_ok=True)
os.makedirs(stereo_output_dir, exist_ok=True)
process_images(stereo_input_dir, stereo_output_dir, 0.05)


## Inpainting Process

Run the script for inpainting left and right eye images or run the function inside notebook.


In [ ]:
stereo_output_dir = dataset_directory + "stereo/"
stereo_postprocess_dir = dataset_directory + "stereo_post/"
os.makedirs(stereo_output_dir, exist_ok=True)
os.makedirs(stereo_postprocess_dir, exist_ok=True)

!python sbs_inpaint_stereoviews.py {stereo_output_dir} {stereo_postprocess_dir}

In [ ]:
import cv2
import numpy as np
import os
import concurrent.futures
from skimage import io, color
from skimage.morphology import binary_dilation, disk


def create_img_mask(color_image, black_pixels_npy_path):
    # print(f"np path {black_pixels_npy_path}")
    # Load the black pixel indexes from the npy file
    black_pixel_indexes = np.load(black_pixels_npy_path)
    
    # Initialize the mask image with zeros (all black)
    mask_image = np.zeros(color_image.shape[:2], dtype=np.uint8)
    
    # Image dimensions
    height, width = color_image.shape[:2]
    
    # Check the shape of the loaded indexes and convert if necessary
    if black_pixel_indexes.ndim == 1:
        # Convert linear indices to 2D coordinates
        y_coords, x_coords = np.divmod(black_pixel_indexes, width)
    else:
        # Assuming the array is already in the form of (y, x) pairs
        y_coords, x_coords = black_pixel_indexes[:, 0], black_pixel_indexes[:, 1]
    
    # Mark the black pixels in the mask image
    mask_image[y_coords, x_coords] = 255  # Mark as white in the mask
    return mask_image
    
def inpaint_image_with_mask(color_image, mask, inpaint_radius=3):
    """
    Inpaints a color image using a mask image.

    Parameters:
    - color_image_path: Path to the color image.
    - mask_image_path: Path to the mask image where white pixels indicate areas to inpaint.
    - inpaint_radius: Radius of a circular neighborhood of each point inpainted that is considered by the algorithm.

    Returns:
    - inpainted_image: The inpainted image as a NumPy array.
    """    
    # Load the color image and mask
    # color_image = cv2.imread(color_image_path)
    # mask = cv2.imread(mask_image_path, cv2.IMREAD_GRAYSCALE)

    # Ensure the mask is binary
    # _, mask_binary = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    # mask_binary = cv2.adaptiveThreshold(mask, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 8)
    mask_binary = cv2.adaptiveThreshold(mask, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 8)
    # Perform inpainting
    print("about to perform inpaint operation")
    # inpainted_image = cv2.inpaint(color_image, mask_binary, inpaint_radius, cv2.INPAINT_TELEA)
    inpainted_image = cv2.inpaint(color_image, mask_binary, 3, cv2.INPAINT_TELEA)

    # Convert image to grayscale
    gray_image = color.rgb2gray(inpainted_image)
    threshold = 0.05  # A value close to 0 to capture black
    binary_mask = gray_image < threshold    
    # Dilate the mask to ensure all black or near black pixels are included
    dilated_mask = binary_dilation(binary_mask, disk(1))    
    inpainted_image = cv2.inpaint(inpainted_image, dilated_mask.astype(np.uint8), 3, cv2.INPAINT_TELEA)
    print("inpaint complete")
    return inpainted_image

def process_single_pair(left_eye_image_path, right_eye_image_path, left_eye_mask_path, right_eye_mask_path, output_dir, save_masks=True):
    # Read the images
    left_eye_image = cv2.imread(left_eye_image_path)
    right_eye_image = cv2.imread(right_eye_image_path)

    if left_eye_image is None or right_eye_image is None:
        print(f"Error: Image not found at {left_eye_image_path} or {right_eye_image_path}")
        return

    # Extract frame number from the image path
    frame_number = os.path.basename(left_eye_image_path).split('leftEye')[1].split('.')[0]

   # Paths for saving masks should use os.path.join to avoid path separator issues
    left_eye_mask_dir = os.path.join(output_dir, 'leftEye')
    right_eye_mask_dir = os.path.join(output_dir, 'rightEye')
    os.makedirs(left_eye_mask_dir, exist_ok=True)
    os.makedirs(right_eye_mask_dir, exist_ok=True)
    
    left_eye_mask_output_path = os.path.join(left_eye_mask_dir, f'leftEyeMask{frame_number}.jpg')
    right_eye_mask_output_path = os.path.join(right_eye_mask_dir, f'rightEyeMask{frame_number}.jpg')
    left_eye_inpaint_output_path = os.path.join(left_eye_mask_dir, f'leftEyeInpaint{frame_number}.jpg')
    right_eye_inpaint_output_path = os.path.join(right_eye_mask_dir, f'rightEyeInpaint{frame_number}.jpg')

    if save_masks:
        # left_eye_mask_path = left_eye_mask_dir + r"/leftEyeMask" + str(frame_number) + '.npy'
        # right_eye_mask_path = right_eye_mask_dir + r"/rightEyeMask" + str(frame_number) + '.npy'
        # print(f"left {left_eye_mask_output_path} right {right_eye_mask_output_path}")
        # Create masks for the black streaks in both left and right eye images
        left_eye_mask = create_img_mask(left_eye_image, left_eye_mask_path)    
        right_eye_mask = create_img_mask(right_eye_image, right_eye_mask_path)    

        left_eye_inpaint = inpaint_image_with_mask(left_eye_image, left_eye_mask)
        right_eye_inpaint = inpaint_image_with_mask(right_eye_image, right_eye_mask)
        # Save the masks
        cv2.imwrite(left_eye_mask_output_path, left_eye_mask)
        cv2.imwrite(right_eye_mask_output_path, right_eye_mask)
        cv2.imwrite(left_eye_inpaint_output_path, left_eye_inpaint)
        cv2.imwrite(right_eye_inpaint_output_path, right_eye_inpaint)
        

    print(f"Processed frame {frame_number}, {left_eye_mask_output_path}")


def process_images(input_dir, output_dir, save_masks=True):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)


    left_path = os.path.join(input_dir, "leftEye")
    right_path = os.path.join(input_dir, "rightEye")
    
    # Get sorted list of image paths
    left_eye_images = sorted([os.path.join(left_path, f) for f in os.listdir(left_path) if f.startswith('leftEye')])
    right_eye_images = sorted([os.path.join(right_path, f) for f in os.listdir(right_path) if f.startswith('rightEye')])
    left_eye_masks = sorted([os.path.join(left_path, f) for f in os.listdir(left_path) if f.startswith('leftEyeMask')])
    right_eye_masks = sorted([os.path.join(right_path, f) for f in os.listdir(right_path) if f.startswith('rightEyeMask')])
    # Process images in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count() - 1) as executor:
        # Submit tasks to the executor
        tasks = {
            executor.submit(process_single_pair, left_eye_image_path, right_eye_image_path, left_eye_mask_path, right_eye_mask_path, output_dir, save_masks): (left_eye_image_path, right_eye_image_path, left_eye_mask_path, right_eye_mask_path)
            for left_eye_image_path, right_eye_image_path, left_eye_mask_path, right_eye_mask_path in zip(left_eye_images, right_eye_images, left_eye_masks, right_eye_masks)
        }

        # Wait for all tasks to complete
        for future in concurrent.futures.as_completed(tasks):
            # You can add error checking here if you like
            try:
                future.result()
            except Exception as e:
                left_eye_image_path, right_eye_image_path = tasks[future]
                print(f"Exception processing {left_eye_image_path} and {right_eye_image_path}: {e}")
        # list(tqdm(concurrent.futures.as_completed(futures), total=len(futures)))
    print("Image processing completed.")


# Setup directories
input_directory = f"{dataset_directory}/stereo/"
output_directory = f"{dataset_directory}/stereo_post/"
os.makedirs(output_directory, exist_ok=True)

# Iterate over the input directory and process pairs
process_images(input_directory, output_directory)
print("Image processing completed.")

In [ ]:
import cv2
import numpy as np
import os

def create_mask_for_black_streaks(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Use adaptive thresholding to better capture the black streaks
    mask = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 8)
    
    # Dilate the mask to include the edges of the black streaks
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.dilate(mask, kernel, iterations=1)
    
    return mask

def inpaint_black_streaks(image, mask):
    # Inpaint the black streaks in the image
    inpainted_image = cv2.inpaint(image, mask, 5, cv2.INPAINT_TELEA)
    
    return inpainted_image

def process_images(input_dir, output_dir, save_masks=False):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if not os.path.exists(os.path.join(output_dir,'leftEye')):
        os.makedirs(os.path.join(output_dir,'leftEye'))
    if not os.path.exists(os.path.join(output_dir,'rightEye')):
        os.makedirs(os.path.join(output_dir,'rightEye'))
    
    if save_masks:
        if not os.path.exists(os.path.join(output_dir,'leftEyeMask')):
            os.makedirs(os.path.join(output_dir,'leftEyeMask'))
        if not os.path.exists(os.path.join(output_dir,'rightEyeMask')):
            os.makedirs(os.path.join(output_dir,'rightEyeMask'))

    left_path = input_dir + "leftEye/"
    right_path = input_dir + "rightEye/"
    left_eye_images = sorted([f for f in os.listdir(left_path) if f.startswith('leftEye')])
    right_eye_images = sorted([f for f in os.listdir(right_path) if f.startswith('rightEye')])

    for left_eye_image_path, right_eye_image_path in zip(left_eye_images, right_eye_images):
        left_eye_image = cv2.imread(os.path.join(left_path, left_eye_image_path))
        right_eye_image = cv2.imread(os.path.join(right_path, right_eye_image_path))

        if left_eye_image is None or right_eye_image is None:
            print(f"Error: Image not found at {os.path.join(input_dir, left_eye_image_path)} or {os.path.join(input_dir, right_eye_image_path)}")
            continue

        # Create masks for the black streaks in both left and right eye images
        left_eye_mask = create_mask_for_black_streaks(left_eye_image)
        right_eye_mask = create_mask_for_black_streaks(right_eye_image)

        # Inpaint the black streaks in both left and right eye images
        # left_eye_post = inpaint_black_streaks(left_eye_image, left_eye_mask)
        # right_eye_post = inpaint_black_streaks(right_eye_image, right_eye_mask)

        frame_number = left_eye_image_path.split('leftEye')[1].split('.')[0]
        left_eye_post_output_path = os.path.join(output_dir + "leftEye/", f'leftEyePost{frame_number}.jpg')
        right_eye_post_output_path = os.path.join(output_dir + "rightEye/", f'rightEyePost{frame_number}.jpg')
        # Save the processed images and masks
        # cv2.imwrite(left_eye_post_output_path, left_eye_post)
        # cv2.imwrite(right_eye_post_output_path, right_eye_post)
        
        if (save_masks):
            left_eye_mask_output_path = os.path.join(output_dir + "leftEyeMask/", f'leftEyeMask{frame_number}.jpg')
            right_eye_mask_output_path = os.path.join(output_dir + "rightEyeMask/", f'rightEyeMask{frame_number}.jpg')
            cv2.imwrite(left_eye_mask_output_path, left_eye_mask)
            cv2.imwrite(right_eye_mask_output_path, right_eye_mask)

        print(f"Processed frame {frame_number}.")

                          
stereo_output_dir = dataset_directory + "stereo_out_frames/"
stereo_postprocess_dir = dataset_directory + "stereo_postprocess_frames/"
os.makedirs(stereo_output_dir, exist_ok=True)
os.makedirs(stereo_postprocess_dir, exist_ok=True)                          
# Example usage
process_images(stereo_output_dir, stereo_postprocess_dir)



## Create Videos from Images

Use ffmpeg to create left and right eye videos. 

**Note** what the appropriate frame rate should be based on your input video.


In [ ]:
left_eye_dir = dataset_directory + "stereo_post/" + "leftEye/"
right_eye_dir = dataset_directory + "stereo_post/" + "rightEye/"
os.makedirs(left_eye_dir, exist_ok=True)
os.makedirs(right_eye_dir, exist_ok=True)
left_eye_dir += "leftEyeInpaint%d.jpg" # "leftEyePost%d.jpg"
right_eye_dir +=  "rightEyeInpaint%d.jpg" # "rightEyePost%d.jpg"

left_eye_vid = dataset_directory + "left_eye.mp4"
right_eye_vid = dataset_directory + "right_eye.mp4"


!ffmpeg -framerate 24 -i {left_eye_dir} -c:v libx264 -pix_fmt yuv420p -vf "fps=24" {left_eye_vid}
!ffmpeg -framerate 24 -i {right_eye_dir} -c:v libx264 -pix_fmt yuv420p -vf "fps=24" {right_eye_vid}



## Merge Videos and Inject Metadata

Combine the left and right eye videos into an SBS video and inject 3D metadata.


In [ ]:
left_eye_vid = dataset_directory + "left_eye.mp4"
right_eye_vid = dataset_directory + "right_eye.mp4"
output_vid = dataset_directory + "output.SBS.mp4"

!ffmpeg -i {left_eye_vid} -i {right_eye_vid} -filter_complex "[0:v][1:v]hstack=inputs=2[v]" -map "[v]" {output_vid}
!ffmpeg -i {output_vid} -vf "scale=2*iw:ih" -c:v libx264 -x264opts "frame-packing=3" -aspect 2:1 {final_video_filename}

In [ ]:
# add audio back
output_vid_audio = dataset_directory + "final.SBS.mp4"
!ffmpeg -i {output_vid} -i {output_audio} -c:v copy -c:a aac -strict experimental {output_vid_audio}

## Upload SBS video to Quest headset

First register device connection, then push file, and lastly force the file system to update without restarting the device.


In [ ]:
!adb devices
!adb push {final_video_filename} /sdcard/Movies/
!adb shell am force-stop com.android.providers.media.module


## Cleanup and Finalization

(Optional) Cleanup temporary files and display/export the final video path.


In [ ]:
# Example cleanup (adjust as needed)

#rgbd frame directories
rgbd_frames = dataset_directory + 'rgbd_in/'
#stereo directories
stereo_output_dir = dataset_directory + "stereo/"
#inpainting directories
stereo_postprocess_dir = dataset_directory + "stereo_post/"

#video directories
left_eye_vid = dataset_directory + "left_eye.mp4"
right_eye_vid = dataset_directory + "right_eye.mp4"
output_vid = dataset_directory + "output.SBS.mp4"

# delete all directories and videos
!rm -rf {rgbd_frames}
!rm -rf {stereo_output_dir}
!rm -rf {stereo_postprocess_dir}
!rm {left_eye_vid} {right_eye_vid} {output_vid}